# MSP: Informacion de ingresos y egresos hospitalarios 
En este notebook se visualizan los datos de ingresos y egresos hospitalarios del Ministerio de Salud Publica (MSP) de Ecuador.
Se utilizan datos de 2015 a 2024, y se visualizan los flujos de pacientes entre provincias, asi como las causas mas comunes de ingreso hospitalario.


In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from keplergl import KeplerGl


In [7]:
msp_data=pd.read_csv("../data/egresos_msp_final.csv")

C:\Users\Kristian Mendoza\AppData\Local\Temp\ipykernel_10172\31262957.py:1: DtypeWarning: Columns (8,14,28) have mixed types. Specify dtype option on import or set low_memory=False.
  msp_data=pd.read_csv("../data/egresos_msp_final.csv")


## Revision pequeña de informacion de las columnas de causas

In [13]:
cau_221_rx=msp_data["cau221rx"].value_counts()
cau_221_rx=cau_221_rx.reset_index()

In [14]:
cau_221_rx_que_deben_renombrarse={"Parto (O80-O84)":"160 Parto (O80-O84)",
                              "Embarazo terminado en aborto (O00-O08)":"155 Embarazo terminado en aborto (O00-O08)",
                              }

In [ ]:
# hacer un mapeo 
msp_data["cau221rx"]=msp_data["cau221rx"].replace(cau_221_rx_que_deben_renombrarse)

## Adjuntar información sobre canton de residencia

In [3]:
inec_parroquias=pd.read_csv("../data/parroquias_cantones_inec.csv")

In [4]:
columns_to_strip_inec=["canton","parroquia"]
for column in columns_to_strip_inec:
    inec_parroquias[column] = inec_parroquias[column].str.strip()

In [5]:
msp_hospitales_with_coords={"HOSPITAL BÁSICO FISCOMISIONAL FRANKLIN TELLO":(-0.9189074317554865, -75.40111236050274),
                            "HOSPITAL DEL DIA MARIANA DE JESUS":(-2.208323243432747, -79.92643781984766),
                            "HOSPITAL BASICO DE ATACAMES - JUAN CARLOS GUASTI":(0.8738746305926328, -79.83894876985617),
                            "HOSPITAL BASICO DE TAISHA - SAN JOSE":(-2.3890035307647115, -77.50822420427274),
                            "HOSPITAL CLAUDIO BENATI DE ZUMBAHUA":(-0.9581614749564258, -78.89699937428749),
}

In [6]:
for hospital, coords in msp_hospitales_with_coords.items():
    lat, lon = coords
    msp_data.loc[msp_data["nombre_centro_de_salud"] == hospital, "latgps"] = lat
    msp_data.loc[msp_data["nombre_centro_de_salud"] == hospital, "longps"] = lon

In [7]:
columns_to_strip_msp=["prov_res","cant_res_std","parr_res_std"]

for column in columns_to_strip_msp:
    msp_data[column].fillna("",inplace=True)
    msp_data[column]=msp_data[column].astype(str)
    msp_data[column] = msp_data[column].str.strip()


C:\Users\Kristian Mendoza\AppData\Local\Temp\ipykernel_10684\1686445103.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  msp_data[column].fillna("",inplace=True)
C:\Users\Kristian Mendoza\AppData\Local\Temp\ipykernel_10684\1686445103.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

In [8]:
inec_cantones=inec_parroquias[["provincia","canton","code_canton"]].drop_duplicates()

In [9]:
# Merge de msp_data con inec_parroquias para obtener codigos de canton
msp_data_canton_res=pd.merge(msp_data,inec_cantones[["provincia","canton","code_canton"]],left_on=["prov_res","cant_res_std"],right_on=["provincia","canton"],how="left").rename(columns={"code_canton":"code_canton_res"})

In [10]:
msp_data_canton_res.drop(columns=["canton_y","provincia_y"],inplace=True)

In [11]:
msp_data_canton_res.rename(columns={"code_canton_y":"code_canton_res","code_canton_x":"code_canton_ubi"},inplace=True)

In [12]:
msp_data_hospitales=msp_data[["nombre_centro_de_salud","latgps","longps"]]

In [13]:
msp_data_hospitales=msp_data_hospitales.drop_duplicates(subset=["nombre_centro_de_salud"])

In [2]:
cantones = gpd.read_file("../data/organizacion-territorial-cantonal/ORGANIZACION_TERRITORIAL_CANTONAL.shp")
inec_parroquias=gpd.read_file("../data/LIMITE_PARROQUIAL_CONALI_CNE_2022/LIMITE_PARROQUIAL_CONALI_CNE_2022/LIMITE_PARROQUIAL_CONALI_CNE_2022.shp")

In [ ]:
i

In [ ]:
# Ver el CRS actual
print(cantones.crs)
# Convertir a lat/lon (EPSG:4326)
cantones = cantones.to_crs(epsg=4326)
cantones["centroid"] = cantones.geometry.centroid
cantones["lat_canton"] = cantones.centroid.y
cantones["lon_canton"] = cantones.centroid.x
cantones["DPA_CANTON"] = "EC" + cantones["DPA_CANTON"].astype(str)
cantones.rename(columns={"DPA_CANTON":"code_canton"},inplace=True)

In [15]:
msp_data.columns

Index(['area_ubi', 'clase', 'tipo', 'entidad_x', 'sector', 'mes_inv',
       'nac_pac', 'nom_pais', 'cod_pais', 'sexo', 'cod_edad', 'edad', 'etnia',
       'prov_res', 'area_res', 'anio_ingr', 'mes_ingr', 'dia_ingr',
       'fecha_ingr', 'anio_egr', 'mes_egr', 'dia_egr', 'fecha_egr',
       'dia_estad', 'con_egrpa', 'esp_egrpa', 'cau_cie10', 'causa3',
       'cap221rx', 'cau221rx', 'cau298rx', 'cant_ubi_std_x', 'parr_ubi_std_x',
       'cant_res_std', 'parr_res_std', 'ruc', 'eod', 'unicodigo_geosalud',
       'nombre_centro_de_salud', 'nivel_de_atencion', 'direccion', 'longps',
       'latgps', 'code_provincia_x', 'provincia', 'code_canton', 'canton',
       'code_parroquia', 'sri_parroquia', 'parroquia'],
      dtype='object')

In [16]:
msp_data_2015=msp_data_canton_res[msp_data_canton_res["anio_ingr"]==2024.0]
#msp_data_2015=msp_data_canton_res.copy()

In [17]:
msp_data_2015_sample = msp_data_2015[["clase","tipo","nac_pac","sexo","edad","etnia","esp_egrpa","nombre_centro_de_salud","code_parroquia","parroquia","code_canton_ubi","latgps","longps","code_canton_res","cau298rx","cau221rx","fecha_ingr"]]

In [18]:
msp_data_2015_sample["fecha_ingr"] = pd.to_datetime(msp_data_2015_sample["fecha_ingr"])


C:\Users\Kristian Mendoza\AppData\Local\Temp\ipykernel_10684\3089088071.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msp_data_2015_sample["fecha_ingr"] = pd.to_datetime(msp_data_2015_sample["fecha_ingr"])


## Flujos DE PACIENTES POR ESPECIALIDAD

In [19]:
#agrupamos pacientes por cantón de residencia, hospital y causa de ingreso

flujos = (
    msp_data_2015_sample.groupby(["clase","code_canton_res", "nombre_centro_de_salud", "esp_egrpa",])
      .size()
      .reset_index(name="conteo")
)


In [20]:
# Merge con cantones (origen)
flujos = flujos.merge(
    cantones[["code_canton", "lat_canton", "lon_canton","DPA_DESCAN"]],
    left_on="code_canton_res",
    right_on="code_canton",
    how="left"
).rename(columns={"lat_canton": "lat_origen", "lon_canton": "lon_origen","DPA_DESCAN":"canton_origen"}).drop(columns=["code_canton"])

In [21]:
# Merge con hospitales (destino)
flujos = flujos.merge(
    msp_data_hospitales,
    on="nombre_centro_de_salud",
    how="left"
).rename(columns={"latgps": "lat_dest", "longps": "lon_dest"})


## FLUJOS DE PACIENTES POR ESPECIALIDAD y FECHA

In [23]:

#agrupamos pacientes por cantón de residencia, hospital y causa de ingreso
flujos_fecha = (
    msp_data_2015_sample
      .groupby(["code_canton_res", "nombre_centro_de_salud", "esp_egrpa", "fecha_ingr","clase"])
      .size()
      .reset_index(name="conteo")
)
# Merge con cantones (origen)
flujos_fecha = flujos_fecha.merge(
    cantones[["code_canton", "lat_canton", "lon_canton","DPA_DESCAN"]],
    left_on="code_canton_res",
    right_on="code_canton",
    how="left"
).rename(columns={"lat_canton": "lat_origen", "lon_canton": "lon_origen","DPA_DESCAN":"canton_origen"}).drop(columns=["code_canton"])
# Merge con hospitales (destino)
flujos_fecha = flujos_fecha.merge(
    msp_data_hospitales,
    on="nombre_centro_de_salud",
    how="left"
).rename(columns={"latgps": "lat_dest", "longps": "lon_dest"})


In [ ]:
flujos_fecha["origen"] = list(zip(flujos_fecha["lon_origen"], flujos_fecha["lat_origen"]))  # (lon, lat)
flujos_fecha["destino"] = list(zip(flujos_fecha["lon_dest"], flujos_fecha["lat_dest"]))      # (lon, lat)
flujos_fecha["fecha_ingr"] = pd.to_datetime(flujos_fecha["fecha_ingr"])
flujos_fecha["fecha_ingr"] = flujos_fecha["fecha_ingr"].dt.strftime("%Y-%m-%dT%H:%M:%S")


In [26]:
from keplergl import KeplerGl

mapa_fechas = KeplerGl(height=600)
mapa_fechas.add_data(data=flujos_fecha, name="Flujos pacientes")
# Aqui van a editar el mapa y luego lo guardan en la siguiente celda
mapa_fechas


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to ../maps/msp_flujos_pacientes_fechas.html!


KeplerGl(data={'Flujos pacientes': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18…

# Flujos con kepler

In [ ]:
mapa_fechas.save_to_html(file_name="../maps/msp_flujos_pacientes_fechas.html")

In [ ]:
# Crear tuplas de coordenadas
flujos["origen"] = list(zip(flujos["lon_origen"], flujos["lat_origen"]))  # (lon, lat)
flujos["destino"] = list(zip(flujos["lon_dest"], flujos["lat_dest"]))      # (lon, lat)


In [ ]:
from keplergl import KeplerGl

mapa = KeplerGl(height=600)
mapa.add_data(data=flujos, name="Flujos Pacientes")
mapa

In [ ]:
mapa.config

# Visualizacion de informacion agregada

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from keplergl import KeplerGl

In [2]:
msp_data=pd.read_csv("../data/egresos_agregado.csv")

In [ ]:
from keplergl import KeplerGl

mapa = KeplerGl(height=600)
mapa.add_data(data=msp_data, name="resumenes Agregados")
mapa


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'resumenes Agregados': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,…

In [6]:
mapa.save_to_html(file_name="../maps/resumenes_agregados.html")

Map saved to ../maps/resumenes_agregados.html!
